In [37]:
import torchaudio
import torch

from torchaudio.utils import download_asset

# get the sample audio
# sample_wav = download_asset("voiptroubleshooter.com/open_speech/american/OSR_us_000_0010_8k.wav")
sample_wav = "./OSR_us_000_0010_8k.wav"

#get the audio's metadata
metadata = torchaudio.info(sample_wav)
print(metadata)

AudioMetaData(sample_rate=8000, num_frames=268985, num_channels=1, bits_per_sample=16, encoding=PCM_S)


In [38]:
# trim the first 3.5 seconds

import numpy as np 
import pandas as pd 
import scipy.io.wavfile as siw
# import DCT from scipt.fftpack 
# returns discrete Fourier transform of real and complex sequence
from scipy.fftpack import dct # Dicrete Cosine Transform

sample_rate, signal = siw.read('./OSR_us_000_0010_8k.wav')
signal = signal [0:int(4*sample_rate)] # keep the first 4 secs

In [39]:
# plot the speech waveform
speech_waveform, sample_rate  = torchaudio.load(sample_wav)
# print(speech_waveform) # tensor
import matplotlib.pyplot as plt

def plot_waveform(waveform, sr):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sr

    figure, axes = plt.subplots(num_channels, 1)
    axes.plot(time_axis, waveform[0], linewidth=1)
    axes.grid(True)
    plt.show(block=False)

# plot_waveform(speech_waveform, sample_rate)
from IPython.display import Audio
Audio(speech_waveform.numpy(), rate=sample_rate)

Signal Step 1: Pre-empasis Filter.

In [40]:
# first order empahsis
pre_ephasis = 0.97 # or 0.95 y(t) = x(t) - (alpha)x(t-1) alpha=0.95
emphasized_signal = np.append(signal[0], signal[1:] - pre_ephasis * signal[:-1])

Step 2: Framing and Slicing

In [42]:
frame_size = 0.025 # 25 ms
# for a 15ms overlap -> 25-15 = 10 ms stride
frame_stride = 0.01 # 10 ms

frame_length, frame_step = frame_size*sample_rate, frame_stride*sample_rate
singal_length = len(emphasized_signal)
# print(singal_length)

frame_length= int(round(frame_length))
# print(frame_length)
frame_step= int(round(frame_step))
# print(frame_step)
num_frames = int(np.ceil(float(np.abs(singal_length- frame_length))/frame_step))
# print(num_frames)

pad_signal_length = num_frames * frame_step + frame_length
z = np.zeros((pad_signal_length - singal_length))

pad_signal = np.append(emphasized_signal, z)

indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
frames = pad_signal[indices.astype(np.int32, copy=False)]